In [2]:
from collections import OrderedDict
import numpy as np
import cv2
import dlib
import imutils
from PIL import Image
import face_recognition

def crop(image):
    image = face_recognition.load_image_file(image)
    face_locations = face_recognition.face_locations(image)
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)
        pil_image.save('image/test/crop.png')

        
def avg(image):
    avg = cv2.mean(image)
    print(avg)
    rec = np.zeros((500, 500,3), np.uint8)
    rec = cv2.rectangle(rec, (0, 0), (500, 500), avg, -1)
    cv2.imwrite("image/test/skin_color.png", rec)

    
def cheek(image):
    CHEEK_IDXS = OrderedDict([("right_cheek", (12,14,47,35))])
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = cv2.imread(image)
    img = imutils.resize(img, width=500)
    overlay = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detections = detector(gray, 0)
    
    for k,d in enumerate(detections):
        shape = predictor(gray, d)
        pts = np.zeros((len(CHEEK_IDXS['right_cheek']), 2), np.int32)
        
        for i,j in enumerate(CHEEK_IDXS['right_cheek']):
            pts[i] = [shape.part(j).x, shape.part(j).y] 
            
        aa = np.array(pts)
        pts = pts.reshape((-1, 1, 2))
        rect = cv2.boundingRect(aa)
        x, y, w, h = rect
        croped = img[y:y + h, x:x + w].copy()
        aa = aa - aa.min(axis=0)
        mask = np.zeros(croped.shape[:2], np.uint8)
        cv2.drawContours(mask, [aa], -1, (255, 255, 255), -1, cv2.LINE_AA)
        dst = cv2.bitwise_and(croped, croped, mask=mask)
        dst = imutils.resize(dst, width=600)
        cv2.imwrite("image/test/dst.png", dst)


def center(image):
    image = Image.open(image)
    width = 500
    height = 500
    left = (width - 100)/2
    top = (height - 100)/2
    right = (width + 100)/2
    bottom = (height + 100)/2
    image = image.crop((left, top, right, bottom))
    image.save('image/test/center.png')


crop('image/yoon.jpg')
cheek('image/test/crop.png')
center('image/test/dst.png')
face_color = cv2.imread('image/test/center.png')
avg(face_color)

(131.37560000000002, 150.4111, 202.9452, 0.0)
